In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 4.7 MB/s 
     |████████████████████████████████| 636 kB 40.3 MB/s 
     |████████████████████████████████| 895 kB 30.1 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 3.3 MB 30.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertForQuestionAnswering

In [ ]:
import json
from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/MyDrive/train-v2.0.json', 'rb') as f:
  squad_dict = json.load(f) 

Mounted at /content/drive


In [ ]:
squad_dict = squad_dict['data']

In [ ]:
squad_dict[0]['paragraphs'][0]['qas']

[{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
  'id': '56be85543aeaaa14008c9063',
  'is_impossible': False,
  'question': 'When did Beyonce start becoming popular?'},
 {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],
  'id': '56be85543aeaaa14008c9065',
  'is_impossible': False,
  'question': 'What areas did Beyonce compete in when she was growing up?'},
 {'answers': [{'answer_start': 526, 'text': '2003'}],
  'id': '56be85543aeaaa14008c9066',
  'is_impossible': False,
  'question': "When did Beyonce leave Destiny's Child and become a solo singer?"},
 {'answers': [{'answer_start': 166, 'text': 'Houston, Texas'}],
  'id': '56bf6b0f3aeaaa14008c9601',
  'is_impossible': False,
  'question': 'In what city and state did Beyonce  grow up? '},
 {'answers': [{'answer_start': 276, 'text': 'late 1990s'}],
  'id': '56bf6b0f3aeaaa14008c9602',
  'is_impossible': False,
  'question': 'In which decade did Beyonce become famous?'},
 {'answers': [{'answer_start': 3

In [ ]:
contexts = []
questions = []
answers = []
for topic in squad_dict:
  for para in topic['paragraphs']:
    for qa in para["qas"]:
      if 'plausible_answers' in qa.keys():
        access = 'plausible_answers'
      else:
        access = 'answers'
      for answer in qa[access]:
        contexts.append(para["context"])
        questions.append(qa["question"])
        answers.append(answer)


In [ ]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    # this means the answer is off by 'n' tokens
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

In [ ]:
add_end_idx(answers, contexts)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
encodings = tokenizer(contexts, questions, truncation = True, padding = True)

In [ ]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [ ]:
add_token_positions(encodings, answers)

In [ ]:
encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items() }

  def __len__(self):
    return len(self.encodings.input_ids)

dataset = SquadDataset(encodings)

In [ ]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [ ]:
for epoch in range(3):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, unit = "batch")
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
